In [ ]:
# GNN-Only Analysis - No Traditional Baseline Needed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
import time
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Load your existing TEC data
with pd.HDFStore('./data/gene_network_data.h5') as store:
    tec = store['TEC']

np_tec_abs = np.abs(tec.to_numpy(copy=True))
gene_names = tec.index.tolist()
n_genes = len(gene_names)
print(f"Loaded {n_genes} genes")

In [ ]:
# GNN Model
class TEC_GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_heads=8, dropout=0.3):
        super(TEC_GNN, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=num_heads, dropout=dropout)
        self.gat2 = GATConv(hidden_dim * num_heads, hidden_dim, heads=1, dropout=dropout)
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.gat1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        embeddings = self.gat2(x, edge_index)
        reconstruction = torch.sigmoid(torch.mm(embeddings, embeddings.t()))
        return embeddings, reconstruction

print("GNN model defined")

In [ ]:
# Prepare data
threshold = 0.75
adj_matrix = (np_tec_abs > threshold).astype(float)
np.fill_diagonal(adj_matrix, 0)

# Edge indices
edge_indices = np.where(adj_matrix > 0)
edge_index = torch.tensor(np.vstack([edge_indices[0], edge_indices[1]]), dtype=torch.long)

# Node features
degrees = np.sum(adj_matrix, axis=1)
mean_corr = np.mean(np_tec_abs, axis=1)
node_features = np.column_stack([degrees, mean_corr])
x = torch.tensor(node_features, dtype=torch.float32)
target_adj = torch.tensor(adj_matrix, dtype=torch.float32)

# Move to device
x = x.to(device)
edge_index = edge_index.to(device)
target_adj = target_adj.to(device)

print(f"Data prepared: {x.shape[0]} nodes, {edge_index.shape[1]} edges")

In [ ]:
# Train GNN
model = TEC_GNN(input_dim=x.size(1)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
start_time = time.time()
losses = []

for epoch in range(200):
    optimizer.zero_grad()
    embeddings, reconstruction = model(x, edge_index)
    loss = F.binary_cross_entropy(reconstruction, target_adj)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    
    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")
print(f"Final loss: {losses[-1]:.6f}")

# Get final embeddings
model.eval()
with torch.no_grad():
    final_embeddings, _ = model(x, edge_index)

In [ ]:
# Clustering analysis
embeddings_np = final_embeddings.cpu().numpy()

# Find optimal clusters
silhouette_scores = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(embeddings_np)
    score = silhouette_score(embeddings_np, labels)
    silhouette_scores.append((k, score))

optimal_k, best_score = max(silhouette_scores, key=lambda x: x[1])
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42)
cluster_labels = final_kmeans.fit_predict(embeddings_np)

unique_labels, counts = np.unique(cluster_labels, return_counts=True)
cluster_sizes = list(zip(unique_labels, counts))
cluster_sizes.sort(key=lambda x: x[1], reverse=True)

print(f"Optimal clusters: {optimal_k}")
print(f"Silhouette score: {best_score:.4f}")
print(f"Cluster sizes: {cluster_sizes[:5]}")

In [ ]:
# Hub analysis
influence_scores = np.linalg.norm(embeddings_np, axis=1)
hub_indices = np.argsort(influence_scores)[::-1][:50]
print(f"Top 50 hub genes identified by GNN")
print(f"Hub influence scores range: {influence_scores[hub_indices[0]]:.3f} to {influence_scores[hub_indices[-1]]:.3f}")

In [ ]:
# Similarity analysis and link prediction
similarity_matrix = cosine_similarity(embeddings_np)
np.fill_diagonal(similarity_matrix, 0)

similarities = similarity_matrix.flatten()
similarities = similarities[similarities > 0]

# Predict high-confidence links
high_sim_indices = np.where(similarity_matrix > 0.8)
predictions = []
for i, j in zip(high_sim_indices[0], high_sim_indices[1]):
    if i < j:
        predictions.append((i, j, similarity_matrix[i, j]))

predictions.sort(key=lambda x: x[2], reverse=True)

print(f"Mean similarity: {np.mean(similarities):.4f}")
print(f"High-confidence predictions (>0.8): {len(predictions)}")
print(f"Top 5 predictions:")
for i, (g1, g2, score) in enumerate(predictions[:5], 1):
    print(f"  {i}. Gene_{g1} - Gene_{g2}: {score:.4f}")

In [ ]:
# Save results for manuscript
results = {
    'gnn_results': {
        'training_time': training_time,
        'final_loss': losses[-1],
        'epochs_trained': len(losses),
        'optimal_clusters': optimal_k,
        'silhouette_score': best_score,
        'cluster_sizes': cluster_sizes,
        'num_hubs': len(hub_indices),
        'mean_similarity': float(np.mean(similarities)),
        'high_confidence_predictions': len(predictions),
        'top_predictions': predictions[:10]
    },
    'metadata': {
        'total_genes': n_genes,
        'threshold': threshold,
        'device': str(device)
    }
}

with open('gnn_only_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("\nGNN ANALYSIS COMPLETE")
print("=" * 40)
print(f"Genes analyzed: {n_genes:,}")
print(f"Training time: {training_time:.1f}s")
print(f"Clusters found: {optimal_k}")
print(f"Hub genes: {len(hub_indices)}")
print(f"Predictions: {len(predictions)}")
print("\nResults saved to gnn_only_results.json")